In [26]:
import os
import pandas as pd
import numpy as np
import math
from sklearn.ensemble import RandomForestClassifier

In [33]:
#read in processed data 
train = pd.read_csv('..\\data\\processed\\trainset.csv')
test = pd.read_csv('..\\data\\processed\\testset.csv')
train.head()


,OutcomeType,gender,hasName,ageWeeks,isDog,isMix,month,weekday,hourOfDay,isFixed,newBreed,newColor
0,Return_to_owner,Male,1,52.0,1,1,2,Wednesday,18,1,Other,Brown/White
1,Euthanasia,Female,1,52.0,0,1,10,Sunday,12,1,Domestic Shorthair Mix,Other
2,Adoption,Male,1,104.0,1,1,1,Saturday,12,1,Pit Bull Mix,Blue/White
3,Transfer,Male,0,3.0,0,1,7,Friday,19,0,Domestic Shorthair Mix,Other
4,Transfer,Male,0,104.0,1,0,11,Friday,12,1,Other,Tan


In [67]:
def fit_randomforest(x_train, y_train, max_features="auto"):
    num_trees = 500
    SEED = 12345
    model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features, random_state=SEED)
    model.fit(x_train, y_train)
    
    return model

In [68]:
categorical = ['gender', 'weekday', 'newBreed', 'newColor']    

# Convert data type as 'category'
train_binary = train.copy()
test_binary = test.copy()
for i in categorical:
    train_binary[i] = train_binary[i].astype('category')
    test_binary[i] = test_binary[i].astype('category')
    
# Create dummy variables
train_binary_dummy = pd.get_dummies(train_binary, columns = categorical)
test_binary_dummy = pd.get_dummies(test_binary, columns = categorical)
test_binary_dummy.drop('OutcomeType', axis=1, inplace=True)
test_binary_dummy.head()


,hasName,ageWeeks,isDog,isMix,month,hourOfDay,isFixed,gender_Female,gender_Male,gender_Unknown,...,newColor_Red/White,newColor_Tan,newColor_Tan/White,newColor_Torbie,newColor_Tortie,newColor_Tricolor,newColor_White,newColor_White/Black,newColor_White/Brown,newColor_White/Tan
0,1,40.0,1,1,10,12,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
1,1,104.0,1,0,7,17,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,52.0,0,1,1,12,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,16.0,1,1,12,18,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
4,1,104.0,1,1,9,17,1,0,1,0,...,0,0,0,0,0,0,1,0,0,0


In [69]:
# train
y_train = train_binary_dummy['OutcomeType']
x_train = train_binary_dummy.drop('OutcomeType', axis=1)
model_rf = fit_randomforest(x_train, y_train)

In [70]:
# test
predicted = pd.DataFrame(model_rf.predict_proba(test_binary_dummy))
predicted.columns = ['Adoption', 'Died', 'Euthanasia', 'Return_to_owner', 'Transfer']
predicted.index += 1
predicted.index.name = 'ID'
predicted.head()

predicted.to_csv('submission.csv')